In [1]:
import Bio
import numpy as np
from Bio import SeqIO
import sys
import csv
import os
import random as random
from functions_collection_copy import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [25]:
#making sketches for the function that will quantify how much better the guide tree was with MST than with Gus!

def collect_guide_tree_cost(matrix_Gus,who_aligned_to_who_Krus,who_aligned_to_who_Prim,MSA_list_Krus,MSA_list_Prim,in_which_MSA_is_it_Krus,in_which_MSA_is_it_Prim,score_matrix,gap_cost):
    min_sum_row_index_Gus=np.argmin(np.sum(matrix_Gus, axis=1))
    cost_guide_tree_Gus=np.sum(matrix_Gus[ min_sum_row_index_Gus, :])
    cost_MST_Krus=0
    cost_MST_Prim=0
    for element in who_aligned_to_who_Krus:
        seq1_nr=element[0]
        seq2_nr=element[1]
        #print("seq1_nr and seq2_nr are: "+str(seq1_nr)+" , "+str(seq2_nr))
        pos_in_MSA_seq1=in_which_MSA_is_it_Krus[seq1_nr][1]
        pos_in_MSA_seq2=in_which_MSA_is_it_Krus[seq2_nr][1]
        #print("pos_in_MSA_seq1,pos_in_MSA_seq2: "+str(pos_in_MSA_seq1)+" , "+str(pos_in_MSA_seq2))
        seq1_from_MSA=[]
        seq2_from_MSA=[]
        j=0
        while j<=len(MSA_list_Krus[0])-1:
            found=MSA_list_Krus[0][j][pos_in_MSA_seq1]
            seq1_from_MSA.append(found)
            j+=1
        j=0
        while j<=len(MSA_list_Krus[0])-1:
            found=MSA_list_Krus[0][j][pos_in_MSA_seq2]
            seq2_from_MSA.append(found)
            j+=1
        union=[]
        k=0
        len_max=max(len(seq1_from_MSA),len(seq1_from_MSA))
        while k<=(len_max-1):
            el1=seq1_from_MSA[k]
            el2=seq2_from_MSA[k]
            tuple_like_zip=[el1,el2]
            union.append(tuple_like_zip)
            k+=1
        cost_element_Krus=compute_cost(union,score_matrix,gap_cost)
        cost_MST_Krus+=(cost_element_Krus)
    for element in who_aligned_to_who_Prim:
        seq1_nr = element[0]
        seq2_nr = element[1]
        #print("seq1_nr and seq2_nr are: "+str(seq1_nr)+" , "+str(seq2_nr))
        pos_in_MSA_seq1 = in_which_MSA_is_it_Prim[seq1_nr][1]
        pos_in_MSA_seq2 = in_which_MSA_is_it_Prim[seq2_nr][1]
        #print("pos_in_MSA_seq1,pos_in_MSA_seq2: "+str(pos_in_MSA_seq1)+" , "+str(pos_in_MSA_seq2))
        seq1_from_MSA = []
        seq2_from_MSA = []
        j = 0
        while j <= len(MSA_list_Prim[0]) - 1:
            found = MSA_list_Prim[0][j][pos_in_MSA_seq1]
            seq1_from_MSA.append(found)
            j += 1
        j = 0
        while j <= len(MSA_list_Prim[0]) - 1:
            found = MSA_list_Prim[0][j][pos_in_MSA_seq2]
            seq2_from_MSA.append(found)
            j += 1
        union = []
        k = 0
        len_max = max(len(seq1_from_MSA), len(seq1_from_MSA))
        while k <= (len_max - 1):
            el1 = seq1_from_MSA[k]
            el2 = seq2_from_MSA[k]
            tuple_like_zip = [el1, el2]
            union.append(tuple_like_zip)
            k += 1
        cost_element_Prim = compute_cost(union, score_matrix, gap_cost)
        cost_MST_Prim += cost_element_Prim
    return(cost_guide_tree_Gus,cost_MST_Krus,cost_MST_Prim)

        

In [26]:
#applying it
repeats=3
nr_of_str=5
len_str=20

random_pile = []
for i in range(0, repeats):
    for j in range(0, nr_of_str):
        a_sequence = []
        for k in range(0, len_str):
            a_sequence.append(random.choice(['a', 'c', 't', 'g']))
        a_sequence = ''.join(a_sequence)
        random_pile.append(a_sequence)
    print(random_pile)
    #running the 3 functions to get the required inputs for collect_guide_tree_cost():
    #kruskal
    matrix_Krus,MSA_list_Krus, total_cost_Krus,in_which_MSA_is_it_Krus,who_aligned_to_who_Krus=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)
    #prim
    matrix_Prim,MSA_list_Prim, total_cost_Prim,in_which_MSA_is_it_Prim,who_aligned_to_who_Prim=new_assembly_Prim_x(random_pile,score_matrix,gap_cost)
    #gusfield
    matrix_Gus,MSA_list_Gus, total_cost_Gus,in_which_MSA_is_it_Gus=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)

    print(total_cost_Krus,total_cost_Prim,total_cost_Gus)
    cost_guide_tree_Gus,cost_MST_Krus,cost_MST_Prim=collect_guide_tree_cost(matrix_Gus,who_aligned_to_who_Krus,who_aligned_to_who_Prim,MSA_list_Krus,MSA_list_Prim,in_which_MSA_is_it_Krus,in_which_MSA_is_it_Prim,score_matrix,gap_cost)
    print(cost_MST_Krus,cost_MST_Prim,cost_guide_tree_Gus)


['aatgggtccagctacctgtt', 'acgccagggtgtgaaactct', 'tatactcgcgtccgcctaac', 'cagcccgagcccaatacaac', 'cctctcgctataacagattg']
























631 584 566
ATTENTION, GUS COST: 178.0
171 171 178.0
['cagcccgagcccaatacaac', 'aatgggtccagctacctgtt', 'acgccagggtgtgaaactct', 'tatactcgcgtccgcctaac', 'cctctcgctataacagattg', 'cgctgaaggacatctcacgg', 'cttcaaacggccgaaaaacg', 'taatggaggggatcgacgcg', 'tccatgctgacctactttct', 'cttaatttagtggaatgtaa']






















































3138 3138 2948
ATTENTION, GUS COST: 420.0
376 376 420.0
['cagcccgagcccaatacaac', 'aatgggtccagctacctgtt', 'acgccagggtgtgaaactct', 'tatactcgcgtccgcctaac', 'cctctcgctataacagattg', 'cgctgaaggacatctcacgg', 'cttcaaacggccgaaaaacg', 'taatggaggggatcgacgcg', 'tccatgctgacctactttct', 'cttaatttagtggaatgtaa', 'agggtaccacttctgcttag', 'atcaagatagcgagaacggg', 'ttaattgaacatgtggcttt', 'cggtctaactaggtgcatat', 'ggacgctatttcccagctgt']




















































































8204 7